In [1]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re
import random

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  

# Importar librerías para pausar la ejecución
from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_1 = pd.read_csv("INFO_PELIS/cortos_2000_2006.csv", index_col = 0)
df_2 = pd.read_csv("INFO_PELIS/cortos_2021.csv", index_col = 0)
df_3 = pd.read_csv("INFO_PELIS/cortos_2022.csv", index_col = 0)
df_4 = pd.read_csv("INFO_PELIS/cortos_2023.csv", index_col = 0)
df_5 = pd.read_csv("INFO_PELIS/peliculas_2021_2023.csv", index_col = 0)
df_6 = pd.read_csv("INFO_PELIS/pelis_2000_2006.csv", index_col = 0)
df_7 = pd.read_csv("INFO_PELIS/pelis_2007_2010.csv", index_col = 0)
df_8 = pd.read_csv("INFO_PELIS/pelis_2011_2013.csv", index_col = 0)
df_9 = pd.read_csv("INFO_PELIS/cortos_2007_2008.csv", index_col = 0)
df_10 = pd.read_csv("INFO_PELIS/cortos_2009_2010.csv", index_col = 0)
df_11 = pd.read_csv("INFO_PELIS/cortos_2011.csv", index_col = 0)
df_12 = pd.read_csv("INFO_PELIS/cortos_2012.csv", index_col = 0)
df_13 = pd.read_csv("INFO_PELIS/cortos_2013.csv", index_col = 0)
df_14 = pd.read_csv("INFO_PELIS/lista_pelis_2014_2019.csv", index_col = 0)
df_15 = pd.read_csv("INFO_PELIS/cortos_2014_2019.csv", index_col = 0)



In [3]:
df_peliculas = pd.concat([df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15], axis=0)

In [4]:
df_peliculas.head()

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID,Titulo
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874,NaN
1,Drama,Short,Nights,2000,3.0,tt0138655,NaN
2,Drama,Short,Caught,2000,8.0,tt0142155,NaN
3,Drama,Short,Smog,2000,3.0,tt0144613,NaN
4,Drama,Short,Safe Journey,2000,NaN,tt0188189,NaN


In [5]:
df_peliculas.tail()

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID,Titulo
175153,NaN,Short,NaN,2019,9.0,tt30625971,Temptation (Eghva)
175154,NaN,Short,NaN,2019,8.0,tt30701899,Hard Way Out
175155,NaN,Short,NaN,2019,2.0,tt30766508,The Battle of the Sunset
175156,NaN,Short,NaN,2019,6.0,tt30796814,Bear Squad
175157,NaN,Short,NaN,2019,5.0,tt31186006,Dream Fast


In [6]:
df_peliculas.to_csv('union_peliculas.csv')

In [7]:
lista_titulos= df_peliculas["Título"].to_list()

In [8]:
lista_años= df_peliculas["Año de estreno"].to_list()

In [9]:
dict_busqueda_rotten = {}
for titulo,año in zip(lista_titulos,lista_años):
    dict_busqueda_rotten[titulo] = año


In [10]:
lista_tipo= df_peliculas["Tipo"].to_list()
cortos= 0
pelis = 0

for elemento in lista_tipo:
    if elemento == "Short":
        cortos += 1
    elif elemento == "Movie":
        pelis +=1

print(cortos,pelis)

#30% pelis y 70%cortos

380488 171253


In [11]:
lista_ID = df_peliculas["ID"].to_list()

In [12]:
lista1=[df_1,df_2,df_3,df_4,df_5,df_6,df_7,df_8,df_9,df_10,df_11,df_12,df_13,df_14,df_15]

suma = 0
for e in lista1:
    suma += len(e)
print("Longitus suma de los data",suma)   
print("Longitud DATAFRAME", len(lista_ID)) 


Longitus suma de los data 551741
Longitud DATAFRAME 551741


In [14]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## CAMBIAR A TUPLA
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}
limpia_actores={}
diccionario_actores = {"Actor":[], "Fehca_nacimiento":[], "Conocido_por:":[], "Que_hace":[], "Premios":[]}

lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    
# Seleccionar nombre de la pelicula
    try:
        diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", 'span[data-testid="hero__primary-text"]').text)
    except:
        diccionario_peliculas["Nombre de la película"].append(f"Error nombre para la pelicula con id: {pelicula}")
        
# Seleccionar puntuacion 
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("css selector",'div[data-testid="hero-rating-bar__aggregate-rating__score"] > span:first-child').text)
        
    except:  
        diccionario_peliculas["Puntuacion_IMDB"].append("--")


    existe_direccion = False
    existe_guion = False

    for elemento in driver.find_elements("css selector",'li[data-testid="title-pc-principal-credit"]'):
 
        try:
            nombre = elemento.find_element("css selector", "span").text
            if nombre == "Dirección":
                existe_direccion = True

                # Seleccionar director
                try:          
                    diccionario_peliculas["Dirección"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                except:
                    diccionario_peliculas["Dirección"].append("DESCONOCIDO") 
            

            elif nombre == "Guión":
                existe_guion = True
            
                # Seleccionar Guionista
                try:          
                    diccionario_peliculas["Guionistas"].append([e.text for e in elemento.find_elements("css selector",'div > ul > li > a') if e.text != ""])
                except:
                    diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
        except:
            continue

    if existe_direccion == False:    
        diccionario_peliculas["Dirección"].append("DESCONOCIDO")
    if existe_guion == False:
        diccionario_peliculas["Guionistas"].append("DESCONOCIDO")
     

    sleep(5)
    # Seleccionar ARGUMENTO
    
    try:      
        diccionario_peliculas["Argumento"].append(driver.find_element("css selector", 'span[data-testid="plot-xl"]').text)
            
    except:
        diccionario_peliculas["Argumento"].append("Desconocido")

    # Seleccionar DURACION        
    try:   
        busqueda_duracion = driver.find_element("css selector",'div.sc-491663c0-3.bdjVSf > div:first-child > ul > li:last-child').text
        if "min" in busqueda_duracion or "h" in busqueda_duracion:
            diccionario_peliculas["Duración"].append(busqueda_duracion)
        else:
            diccionario_peliculas["Duración"].append("--")
    except:
            diccionario_peliculas["Duración"].append("--")

    # Buscar los 10 primeros actores    
    # try:
    
    # except:

    #     diccionario_peliculas["10actores"].append("--")




### REVISAR:
# Lista de tuplas



In [15]:
diccionario_peliculas

{'Puntuacion_IMDB': ['7,6'],
 'Dirección': [['Kalman Apple']],
 'Guionistas': [['Kalman Apple', 'Anton Chekhov']],
 'Argumento': ["Three actors and a director perform Chekhov's The Bear on a New York City bus (actually, a succession of buses, as they keep getting thrown off). The other riders on the final bus get progressively more interested."],
 'Duración': ['28min'],
 'Nombre de la película': ['Speed for Thespians'],
 '10actores': []}

In [28]:
#Busqueda actores

driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

# dict_actores = {"actor":[],"url":[]}
dict_actores={}


lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(random.randint(5,12))

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):
        actor =[e for e in  elemento.find_elements("css selector", "div > div > div > div > a") if e.text != "" and e.text != "\n"]  
 
        for element in actor[0:10]:
            nombre = element.text
           
            if nombre not in dict_actores.keys():
                dict_actores[nombre]  = element.get_attribute("href")

                driver.find_element("css selector",'section[data-testid="title-cast"]')


                

                


    # try:
    
    # except:

    #     diccionario_peliculas["10actores"].append("--")

AttributeError: 'WebDriver' object has no attribute 'element'

In [24]:
print(dict_actores)


{'Jamie Harris': 'https://www.imdb.com/name/nm0364807/?ref_=tt_cl_t_1', 'Camilla Enders': 'https://www.imdb.com/name/nm0256813/?ref_=tt_cl_t_2', 'Denis Holmes': 'https://www.imdb.com/name/nm0391799/?ref_=tt_cl_t_3', 'Layla Alexander': 'https://www.imdb.com/name/nm0018916/?ref_=tt_cl_t_4', 'Olga Alexandrova': 'https://www.imdb.com/name/nm14869118/?ref_=tt_cl_t_5', 'Michele Ammon': 'https://www.imdb.com/name/nm1994376/?ref_=tt_cl_t_6', 'Tyler Andrews': 'https://www.imdb.com/name/nm14869119/?ref_=tt_cl_t_7', 'Dann R. Black': 'https://www.imdb.com/name/nm14869117/?ref_=tt_cl_t_8', 'Valerie Bodoy': 'https://www.imdb.com/name/nm14869122/?ref_=tt_cl_t_9', 'Jean Pierre Brax': 'https://www.imdb.com/name/nm14869114/?ref_=tt_cl_t_10'}


In [20]:
##Busqueda info actores 
driver = webdriver.Chrome()

for link_actor in dict_actores["url"]:

    link = link_actor

    driver.get(link)

    driver.maximize_window()

    sleep(5)

    # Aceptar las cookies PAG IMDB
    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(5)

    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd"}
  (Session info: chrome=122.0.6261.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF66A4EAD22+56930]
	(No symbol) [0x00007FF66A45F622]
	(No symbol) [0x00007FF66A3142E5]
	(No symbol) [0x00007FF66A3598ED]
	(No symbol) [0x00007FF66A359A2C]
	(No symbol) [0x00007FF66A39A967]
	(No symbol) [0x00007FF66A37BCDF]
	(No symbol) [0x00007FF66A3981E2]
	(No symbol) [0x00007FF66A37BA43]
	(No symbol) [0x00007FF66A34D438]
	(No symbol) [0x00007FF66A34E4D1]
	GetHandleVerifier [0x00007FF66A866AAD+3709933]
	GetHandleVerifier [0x00007FF66A8BFFED+4075821]
	GetHandleVerifier [0x00007FF66A8B817F+4043455]
	GetHandleVerifier [0x00007FF66A589756+70

In [46]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)

actores = {"actor":[]}
url_actores={"url":[]}

lista_ID

for pelicula in lista_ID[130:131]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    for elemento in driver.find_elements("css selector",'section[data-testid="title-cast"]'):

        print(elemento.text.strip("\n"))

#         actores["actor"].append(elemento.text.strip("\n"))
#         url_actores["url"].append(elemento.get_attribute('href'))


# print(actores)
# print("_______")
# print(url_actores)

Reparto principal
Editar
Jamie Harris
Smirnov
Camilla Enders
Popova
Denis Holmes
Luka
Layla Alexander
Passenger
Olga Alexandrova
Girlfriend
Michele Ammon
Irritated Passenger
Tyler Andrews
Boyfriend
Dann R. Black
Telephone repairman
Valerie Bodoy
Homegirl
Jean Pierre Brax
Times reader
Erin Cahill
Sister(as Erin J. Cahill)
Cindy Cheung
Nurse
Justin Christopher
Skater kid
Lee Lee Core
Sipping straw girl
Jamie Elles
Goth girl
John R. Hartmann
Businessman
Betty Hudson
Friend with question lady
Stefanie Imhoff
Sister
Dirección
Kalman Apple
Guión
Kalman AppleAnton Chekhov
Todo el reparto y equipo
Producción, taquilla y más en IMDbPro


In [12]:
df_peliculas.head(25)

,Genero,Tipo,Título,Año de estreno,Mes de estreno,ID,Titulo
0,Drama,Short,Chateaubriand - Cabeça de Paraíba,2000,12.0,tt0115874,NaN
1,Drama,Short,Nights,2000,3.0,tt0138655,NaN
2,Drama,Short,Caught,2000,8.0,tt0142155,NaN
3,Drama,Short,Smog,2000,3.0,tt0144613,NaN
4,Drama,Short,Safe Journey,2000,NaN,tt0188189,NaN
5,Drama,Short,Among Others,2000,6.0,tt0191782,NaN
6,Drama,Short,Ruben,2000,6.0,tt0194319,NaN
7,Drama,Short,O Pedido de Emprego,2000,2.0,tt0195979,NaN
8,Drama,Short,Soul Cages,2000,10.0,tt0202592,NaN
9,Drama,Short,Shadows,2000,3.0,tt0201919,NaN


In [20]:
dict_busqueda_rotten

{'Chateaubriand - Cabeça de Paraíba': 2000,
 'Nights': 2012,
 'Caught': 2013,
 'Smog': 2009,
 'Safe Journey': 2000,
 'Among Others': 2000,
 'Ruben': 2012,
 'O Pedido de Emprego': 2000,
 'Soul Cages': 2000,
 'Shadows': 2013,
 'Short': 2003,
 'The Good Doctor': 2013,
 'Legs Apart': 2000,
 'The Third Lie': 2000,
 'When the Time Comes': 2011,
 'Cirque du Soleil: Journey of Man': 2000,
 'Quiero ser (I want to be...)': 2000,
 'Somewhere Else': 2011,
 'John Henry': 2010,
 'The Window': 2013,
 'Der Apfel': 2000,
 'Flamenco Shoe': 2000,
 'Into My Arms': 2009,
 'Broken': 2013,
 'Coven': 2000,
 'Flying Over Mother': 2000,
 'A Drogaria': 2000,
 'Ghosthunter': 2000,
 'Friday Night, NYC': 2000,
 'Um Dia na Vida': 2000,
 'Angels in a Cage': 2000,
 'Olhos Mortos': 2000,
 'Château de sable': 2000,
 "Olivia's Story": 2000,
 'Plus rien...': 2000,
 'Plunge': 2011,
 'Lucky': 2013,
 'Desire': 2013,
 'Kovat miehet': 2000,
 'Kuppet': 2000,
 'O Céu Que Nos Impede': 2000,
 'Destiny Stalled': 2000,
 'The Book of

In [15]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(3)

puntuacion_tomatoes = {"Rotten Tomatoes":[]}
contador = 0 
lista_titulos

while contador < 3:
    for titulo, año in dict_busqueda_rotten.items():
        
        busqueda = f"puntuacion Rotten Tomatoes del {titulo} del año {año}"
        print(busqueda)
    

        sleep(random.randint(3,6))

        try:
            #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
            driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)

            sleep(5)
            try:
                puntuacion_tomatoes["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
                driver.back()
            except:
                puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
                puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
                puntuacion_tomatoes["Rotten Tomatoes"].append(puntuacion.group())

            driver.back()
        
        except:
            puntuacion_tomatoes["Rotten Tomatoes"].append("No existe puntuacion")
        driver.back()

        contador +=1




## MODIFICAR:
#  Se tiene que añadir al diccionario de pelis y convertir en TUPLA


puntuacion Rotten Tomatoes del Chateaubriand - Cabeça de Paraíba del año 2000
puntuacion Rotten Tomatoes del Nights del año 2012
puntuacion Rotten Tomatoes del Caught del año 2013
puntuacion Rotten Tomatoes del Smog del año 2009
puntuacion Rotten Tomatoes del Safe Journey del año 2000
puntuacion Rotten Tomatoes del Among Others del año 2000
puntuacion Rotten Tomatoes del Ruben del año 2012


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=122.0.6261.71)
Stacktrace:
	GetHandleVerifier [0x00007FF723C5AD22+56930]
	(No symbol) [0x00007FF723BCF622]
	(No symbol) [0x00007FF723A842E5]
	(No symbol) [0x00007FF723A61D4C]
	(No symbol) [0x00007FF723AF23F7]
	(No symbol) [0x00007FF723B07891]
	(No symbol) [0x00007FF723AEBA43]
	(No symbol) [0x00007FF723ABD438]
	(No symbol) [0x00007FF723ABE4D1]
	GetHandleVerifier [0x00007FF723FD6AAD+3709933]
	GetHandleVerifier [0x00007FF72402FFED+4075821]
	GetHandleVerifier [0x00007FF72402817F+4043455]
	GetHandleVerifier [0x00007FF723CF9756+706710]
	(No symbol) [0x00007FF723BDB8FF]
	(No symbol) [0x00007FF723BD6AE4]
	(No symbol) [0x00007FF723BD6C3C]
	(No symbol) [0x00007FF723BC68F4]
	BaseThreadInitThunk [0x00007FFFA970257D+29]
	RtlUserThreadStart [0x00007FFFAA80AA58+40]


In [16]:
print(puntuacion_tomatoes["Rotten Tomatoes"])

['No existe puntuacion', 'No existe puntuacion', '36%', 'No existe puntuacion', 'No existe puntuacion', 'No existe puntuacion', 'No existe puntuacion']


In [26]:
for id_pelicula in lista_ID[130:133]:
    
    url= f"https://www.imdb.com/title/{id_pelicula}"

    sleep(random.randint(5,12))
    llamada_pelicula = requests.get(url)
    print(llamada_pelicula.content)
    print(url)

b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n</body>\r\n</html>\r\n'
https://www.imdb.com/title/tt0269899
b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n</body>\r\n</html>\r\n'
https://www.imdb.com/title/tt0270026
b'<html>\r\n<head><title>403 Forbidden</title></head>\r\n<body>\r\n<center><h1>403 Forbidden</h1></center>\r\n</body>\r\n</html>\r\n'
https://www.imdb.com/title/tt0270717
